In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [90]:
import pickle

# Load data from file
with open('data_all_outer.pkl', 'rb') as f:
    data = pickle.load(f)

In [91]:
# get the absolute path of the current working directory
cwd = os.path.abspath(os.getcwd())

# go up one level in the directory tree
parent_dir = os.path.abspath(os.path.join(cwd, os.pardir))

# go into the 'data' directory
data_dir = os.path.join(parent_dir, 'data')
datafolder = data_dir + '\\' #'./data/' 
co2 = pd.read_csv(datafolder + 'annual-co-emissions-by-region.csv')

In [126]:
# Only keep countries that have info about CO2
data_viz = data[data.country.isin(co2.Entity)]

In [127]:
# Extract interesting feature based on Milestone 1
data_viz = data_viz[['dt_year', 'country','co2emissions_from_fuel_combustion_in_millions_of_tonnes', 
      'total_government_revenue_proportion_of_gdp_perc',  'fdi_inflows_millionusd',
      'annual_growth_rate_perc',
     'proportion_of_population_with_primary_reliance_on_clean_fuels_a',
      'renewable_energy_share_on_the_total_energy_consumption',
      'Annual CO₂ emissions (zero filled)',
     'GDP (constant 2015 US$)', 'mortality_rate_perc',
         'PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total)',
         'Literacy rate, adult total (% of people ages 15 and above)']]# 'Literacy rate, adult total (% of people ages 15 and above)']

In [128]:
data_viz

,dt_year,country,co2emissions_from_fuel_combustion_in_millions_of_tonnes,total_government_revenue_proportion_of_gdp_perc,fdi_inflows_millionusd,annual_growth_rate_perc,proportion_of_population_with_primary_reliance_on_clean_fuels_a,renewable_energy_share_on_the_total_energy_consumption,Annual CO₂ emissions (zero filled),GDP (constant 2015 US$),mortality_rate_perc,"PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total)","Literacy rate, adult total (% of people ages 15 and above)"
0,2002,Afghanistan,NaN,NaN,0.68000,0.420344,8.0,37.83,1340995.0,7.228792e+09,NaN,NaN,NaN
1,2003,Afghanistan,NaN,0.120955,50.00000,0.105249,9.0,36.66,1559602.0,7.867259e+09,NaN,NaN,NaN
2,2004,Afghanistan,NaN,0.176214,57.80000,-0.014901,11.0,44.24,1237247.0,7.978511e+09,NaN,NaN,NaN
3,2005,Afghanistan,NaN,0.180929,186.90000,0.036212,12.0,33.88,1889507.0,8.874476e+09,0.409,100.0,NaN
4,2006,Afghanistan,NaN,0.302234,271.00000,0.053648,14.0,31.89,2159318.0,9.349917e+09,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5171,2017,Zimbabwe,9.66,0.139007,371.81050,0.031871,30.0,82.46,9596071.0,2.106128e+10,NaN,100.0,NaN
5172,2018,Zimbabwe,11.69,0.132036,349.41958,0.033616,30.0,80.23,11795478.0,2.207733e+10,NaN,NaN,NaN
5173,2019,Zimbabwe,11.06,NaN,744.63720,-0.095940,30.0,81.50,11114607.0,2.072084e+10,0.284,NaN,NaN
5174,2020,Zimbabwe,NaN,NaN,280.00000,-0.006748,30.0,NaN,10607897.0,1.942605e+10,NaN,NaN,NaN


In [135]:
# Interpolate between measurement
data_viz = data_viz.groupby('country').apply(pd.DataFrame.interpolate)

In [136]:
data_viz

,dt_year,country,co2emissions_from_fuel_combustion_in_millions_of_tonnes,total_government_revenue_proportion_of_gdp_perc,fdi_inflows_millionusd,annual_growth_rate_perc,proportion_of_population_with_primary_reliance_on_clean_fuels_a,renewable_energy_share_on_the_total_energy_consumption,Annual CO₂ emissions (zero filled),GDP (constant 2015 US$),mortality_rate_perc,"PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total)","Literacy rate, adult total (% of people ages 15 and above)"
0,2002,Afghanistan,NaN,NaN,0.68000,0.420344,8.0,37.83,1340995.0,7.228792e+09,NaN,NaN,NaN
1,2003,Afghanistan,NaN,0.120955,50.00000,0.105249,9.0,36.66,1559602.0,7.867259e+09,NaN,NaN,NaN
2,2004,Afghanistan,NaN,0.176214,57.80000,-0.014901,11.0,44.24,1237247.0,7.978511e+09,NaN,NaN,NaN
3,2005,Afghanistan,NaN,0.180929,186.90000,0.036212,12.0,33.88,1889507.0,8.874476e+09,0.40900,100.0,NaN
4,2006,Afghanistan,NaN,0.302234,271.00000,0.053648,14.0,31.89,2159318.0,9.349917e+09,0.40280,100.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5171,2017,Zimbabwe,9.66,0.139007,371.81050,0.031871,30.0,82.46,9596071.0,2.106128e+10,0.29750,100.0,88.69342
5172,2018,Zimbabwe,11.69,0.132036,349.41958,0.033616,30.0,80.23,11795478.0,2.207733e+10,0.29075,100.0,88.69342
5173,2019,Zimbabwe,11.06,0.132036,744.63720,-0.095940,30.0,81.50,11114607.0,2.072084e+10,0.28400,100.0,88.69342
5174,2020,Zimbabwe,11.06,0.132036,280.00000,-0.006748,30.0,81.50,10607897.0,1.942605e+10,0.28400,100.0,88.69342


In [137]:
# Extract interesting feature based on Milestone 1
rename_dict = {'dt_year': 'Year', 'co2emissions_from_fuel_combustion_in_millions_of_tonnes': 'CO2_fuel_combustion', 
      'total_government_revenue_proportion_of_gdp_perc':'Revenue',  'fdi_inflows_millionusd':'FDI',
      'annual_growth_rate_perc':'Growth_rate',
     'proportion_of_population_with_primary_reliance_on_clean_fuels_a':'Primary_reliance_on_clean_fuels',
      'renewable_energy_share_on_the_total_energy_consumption':'Renewable_energy_share',
      'Annual CO₂ emissions (zero filled)':'CO2_emission',
     'GDP (constant 2015 US$)':'GDP', 'mortality_rate_perc':'Mortality_rate',
         'PM2.5 air pollution, population exposed to levels exceeding WHO guideline value (% of total)':'Air pollution',
         'Literacy rate, adult total (% of people ages 15 and above)':'Literacy rate'}

In [140]:
data_viz = data_viz.rename(columns=rename_dict)

In [141]:
data_viz.to_pickle("./data_viz.pkl")

,Year,country,CO2_fuel_combustion,Revenue,FDI,Growth_rate,Primary_reliance_on_clean_fuels,Renewable_energy_share,CO2_emission,GDP,Mortality_rate,Air pollution,Literacy rate
0,2002,Afghanistan,NaN,NaN,0.68000,0.420344,8.0,37.83,1340995.0,7.228792e+09,NaN,NaN,NaN
1,2003,Afghanistan,NaN,0.120955,50.00000,0.105249,9.0,36.66,1559602.0,7.867259e+09,NaN,NaN,NaN
2,2004,Afghanistan,NaN,0.176214,57.80000,-0.014901,11.0,44.24,1237247.0,7.978511e+09,NaN,NaN,NaN
3,2005,Afghanistan,NaN,0.180929,186.90000,0.036212,12.0,33.88,1889507.0,8.874476e+09,0.40900,100.0,NaN
4,2006,Afghanistan,NaN,0.302234,271.00000,0.053648,14.0,31.89,2159318.0,9.349917e+09,0.40280,100.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5171,2017,Zimbabwe,9.66,0.139007,371.81050,0.031871,30.0,82.46,9596071.0,2.106128e+10,0.29750,100.0,88.69342
5172,2018,Zimbabwe,11.69,0.132036,349.41958,0.033616,30.0,80.23,11795478.0,2.207733e+10,0.29075,100.0,88.69342
5173,2019,Zimbabwe,11.06,0.132036,744.63720,-0.095940,30.0,81.50,11114607.0,2.072084e+10,0.28400,100.0,88.69342
5174,2020,Zimbabwe,11.06,0.132036,280.00000,-0.006748,30.0,81.50,10607897.0,1.942605e+10,0.28400,100.0,88.69342
